In [ ]:
all_dates_df = df.groupBy("id").agg(
    F.date_trunc("mm", F.max(F.to_date("date", "dd/MM/yyyy"))).alias("max_date"),
    F.date_trunc("mm", F.min(F.to_date("date", "dd/MM/yyyy"))).alias("min_date")
).select(
    "id",
    F.expr("sequence(min_date, max_date, interval 1 month)").alias("date")
).withColumn(
    "date", F.explode("date")
).withColumn(
    "date",
    F.date_format("date", "dd/MM/yyyy")
)

In [ ]:
from pyspark.sql import Window
window_spec = Window.partitionBy("ID").orderBy("DATE")

next_date = coalesce(lead("DATE", 1). \
            over(window_spec), col("DATE") + \
                       expr("interval 1 month"))

In [ ]:
window_oil = Window.partitionBy("date")
next_day = coalesce(lead("date", 1).over(window=window_oil), \
                    col('date') + expr('interval 1 day'))

In [ ]:
end_date_range = next_day - expr("interval 1 day")

In [ ]:
oil_data.withColumn("Ranges", sequence(col("date"), end_date_range, 
                                 expr("interval 1 day"))).show()

In [ ]:
oil_data.withColumn("Ranges", sequence(col("date"), end_date_range, 
                                 expr("interval 1 day")))\
  .withColumn("date_now", explode("Ranges")).show(20)

In [ ]:
oil_data.withColumn("Ranges", sequence(col("date"), end_date_range, 
                                 expr("interval 1 day")))\
  .withColumn("date", explode("Ranges"))\
  .withColumn("date", date_format("date", "dd/MM/yyyy"))\
  .drop("Ranges").show(truncate=False)

In [1]:
def get_date_df():
   initial_date = datetime.date(2018, 1, 1)
   days = 365
   one_day = datetime.timedelta(days=1)
   all_days = [{"date": initial_date + i * one_day} for i in range(days)]
   return spark.createDataFrame(Row(**x) for x in all_days)
date_df = get_date_df()
date_df.show(5)

NameError: name 'datetime' is not defined

Following code worked in creating the date_series as planned

In [ ]:
data_date_series = oil_data.select(date_trunc("mm", max(to_date("date", "dd/MM/yyyy"))).\
            alias("max_date"),
            date_trunc("mm", min(to_date("date", "dd/MM/yyyy"))). \
            alias("min_date")). \
    select(expr("sequence(min_date, max_date, interval 1 day)").alias("date_seq")). \
        withColumn("date_new",explode("date_seq")). \
        withColumn("date_form",date_format("date_new", "yyyy-MM-dd"))